In [1]:
# Packages
using Flux, CairoMakie

In [2]:
# Test data
f = x-> cos(x)+sin(x)
x_train_range = range(-2pi,stop=2pi,step=pi/8)
x_test_range = range(0,stop=2pi,step=(pi/4))

x_train = hcat(x_train_range...)
x_test = hcat(x_test_range...)

y_train = map(f,x_train)
y_test = map(f,x_test)

InputDim = size(x_train)[1]
Nh = 60
OutputDim = size(x_train)[1]
Layers = 1
activation = sigmoid

display(y_train)

1×33 Matrix{Float64}:
 1.0  1.30656  1.41421  1.30656  1.0  …  -1.55431e-15  0.541196  1.0

In [3]:
model = Chain(Dense(1,Nh,sigmoid),Dense(Nh,1))|>f64


Chain(
  Dense(1 => 60, σ),                    # 120 parameters
  Dense(60 => 1),                       # 61 parameters
)                   # Total: 4 arrays, 181 parameters, 1.664 KiB.

In [4]:
# Prediction and loss
y_train_hat =  model(x_train)
y_test_hat = model(x_test)

Loss(model,x,y) = Flux.mse(y,model(x));
loss = Loss(model,x_train,y_train)

1.8894614507635186

In [5]:
include("normNTK.jl")

kernel (generic function with 1 method)

In [6]:
for i in 1:length(Flux.params(model))
    display(Flux.params(model)[i])
end


60×1 Matrix{Float64}:
  0.2063131481409073
 -0.18272905051708221
  0.10938817262649536
 -0.17600181698799133
 -0.09328969568014145
  0.09143746644258499
 -0.1269202083349228
 -0.027840018272399902
  0.05913287773728371
  0.10363483428955078
  ⋮
  0.2180255800485611
  0.07881265878677368
 -0.20490403473377228
  0.23591765761375427
  0.17261265218257904
 -0.2956998944282532
 -0.15354862809181213
 -0.22186002135276794
  0.24094000458717346

60-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

1×60 Matrix{Float64}:
 -0.208386  -0.10712  -0.0742964  …  0.00439286  0.122419  -0.0628265

1-element Vector{Float64}:
 0.0

In [168]:
# for i = 1:length(Flux.params(model))
#     myParam = Flux.params(model)[i]
    
#     if isa(myParam, Vector)
#         myParam .= 1/sqrt(length(myParam)) * myParam
#         display(length(myParam))
#     elseif isa(myParam, Matrix) && size(myParam)[2] == 1
#         rows = size(myParam)[1]
#         myParam .= 1/sqrt(rows) * myParam
#         display(length(myParam))
#     elseif isa(myParam, Matrix) && size(myParam)[1] == 1
#         column = size(myParam)[2]
#         myParam .= 1/sqrt(column) * myParam
#         display(length(myParam))
       
#     end
# end



60

60

60

1

In [7]:
K = kernel(model,x_train)
λ = eigen(K).values

DimensionMismatch: DimensionMismatch: array could not be broadcast to match destination

In [12]:
# Training
train_data = [(x_train,y_train)]
opt = Descent(1/λ[end])


Nepoch = 500
Process_Loc = [100]
Process = []
for epoch=1:Nepoch
    Flux.train!(Loss, model, train_data, opt)
    if epoch in Process_Loc
        push!(Process,model(x_train))
    end
end

Process = hcat(Process)

y_train_hat =  model(x_train)



UndefVarError: UndefVarError: `λ` not defined

In [39]:
model[2].bias

1-element Vector{Float64}:
 0.0600645996507981

In [10]:
# loss update 
loss_updt = round(Loss(model,x_train,y_train),digits=5)

1.88946

In [13]:
# Plotting

fig = Figure(resolution = (1400, 600))
ax1 = Axis(fig[1,1],title="Data & Estimation, Cos(x)+Sin(x)")
# ax2 = Axis(fig[1,2],title="Eigenvalues; Nodes = $Nh, Layers = $Layers",yscale=log10)
# s1 = lines!(ax1,x_train[:],y_train[:],label="Actual Data")
# s2 = scatter!(ax1,x_train[:],y_train_hat[:],label="Final Fit (Loss=$loss_updt, $Nepoch Epochs)",color=:red)

for P in 1:length(Process)
    P_num = Process_Loc[P]
    scatter!(ax1,x_train[:],Process[P][:],label="Fitted Data After $P_num Epochs")
end

# scatter!(ax2,1:length(λ),λ[:])
# Create a legend for ax1 with the specified scatter plots
axislegend(ax1,position=:ct)
fig

UndefVarError: UndefVarError: `Process` not defined